
##

## Loading Model from h5

In [23]:
import keras
#from tensorflow.keras.models import model_from_json

In [4]:
# Open and read uploaded json file
json_file = open('model/sage_maker_model/cnn5_model3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


loaded_model = keras.models.model_from_json(loaded_model_json)
loaded_model.load_weights('model/sage_maker_model/cnn5_model_weights3.h5')

In [11]:
new_model = keras.models.load_model('model/converted_keras/keras_model.h5') 

ImportError: Keras requires TensorFlow 2.2 or higher. Install TensorFlow via `pip install tensorflow`

In [19]:
new_model.save_weights('model/sage_maker_model/cnn5_model_weights.h5')
model_json = new_model.to_json()
with open("cnn5_model.json", "w") as json_file:
    json_file.write(model_json)

## Run in SagaMaker notebook instance

In [ ]:
import boto3, re
from sagemaker import get_execution_role
role = get_execution_role()
import keras
from keras.models import model_from_json

## Get Model weights

In [ ]:
JSON_LOCATION = './keras_model/example_model.json'
WEIGHTS_LOCATION = './keras_model/example_model_weights.h5'

if MODEL_LOCATION!='': #if your model is saved as a .h5 file only
    from keras.models import load_model
    model = load_model(MODEL_LOCATION) #load the model
    print("loaded model from MODEL_LOCATION")
    
elif JSON_LOCATION!='': 
    from keras.models import model_from_json
    json_file = open(JSON_LOCATION, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(WEIGHTS_LOCATION)
    print("loaded model from JSON_LOCATION and WEIGHTS_LOCATION")

### Convert model to AWS Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
import sagemaker
import boto3, re
from sagemaker import get_execution_role
def convert_model_to_aws(loaded_model):
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    tf.compat.v1.disable_eager_execution()
    
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    # Build the Protocol Buffer SavedModel at 'export_dir'
    builder = builder.SavedModelBuilder(export_dir)
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

    with tf.compat.v1.Session() as sess:
        init = tf.compat.v1.global_variables_initializer()
        sess.run(init)
        # Save the meta graph and variables
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    #create a tarball/tar file and zip it
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_model_to_aws(model)

## Upload the model to S3

In [ ]:
# Uploading the model zip file to a default s3 bucket
import sagemaker
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

## Deploy the model

In [ ]:
# Get the IAM role and specify the framework version
role = get_execution_role()
framework_version = '2.5' 

# Create Sagemaker model
!touch train.py
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = framework_version,
                                  entry_point = 'train.py')
# create endpoint - can take 5-10min
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

## EndPoint predictions

In [20]:

import numpy as np
import cv2

#Load Image
path = 'data/train/A/e7eb0ce8-5543-11ed-b2a3-f4ee08c64933.jpg'
image = cv2.imread(path)

# resize the image to a 224x224 with the same strategy as in TM2:
imgS = cv2.resize(image, (224, 224))

# turn the image into a numpy array
image_array = np.asarray(imgS)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1
print(normalized_image_array.shape)


# Load the image into the array
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
data[0] = normalized_image_array
print(data.shape)

# run the inference
predictions = loaded_model.predict(data[0])

(224, 224, 3)
(1, 224, 224, 3)


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_4/sequential_1/model1/Conv1_pad/Pad' defined at (most recent call last):
    File "c:\Users\u632981\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\u632981\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\u632981\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\u632981\AppData\Roaming\Python\Python310\site-packages\traitlets\config\application.py", line 982, in launch_instance
      app.start()
    File "C:\Users\u632981\AppData\Roaming\Python\Python310\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\u632981\AppData\Roaming\Python\Python310\site-packages\tornado\platform\asyncio.py",

In [15]:
LABELS = ["A", "B", "C", "D"]
LABELS[np.argmax(predictions)]

'A'

In [ ]:
## Predictions


endpoint_name = 'tensorflow-inference-2022-09-07-13-25-05-023'
photo = 'img-8777.jpeg'

label_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

runtime = boto3.client('runtime.sagemaker')
img = Image.open(photo)
image_array = np.array(img)

predictor = sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name)
result = predictor.predict(data=image_array.reshape(1,32,32,3))
print(label_names[np.argmax(result['predictions'])])